In [ ]:
!pip install music21 keras tensorflow numpy

In [ ]:
import glob
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint
import zipfile
from google.colab import drive
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

## Data Handling

> Loading MIDIs, Extracting Notes, Dataset Preparation


In [ ]:
midi_folder_path = '/content/drive/MyDrive/Midis'

if os.path.exists(midi_folder_path):
    print(f"Files found in: {midi_folder_path}")
else:
    print("MIDI folder not found.")

print("\nExtracted MIDI files:")
midi_files = glob.glob(os.path.join(midi_folder_path, '*.mid'))
if midi_files:
    for midi_file in midi_files:
        print(os.path.basename(midi_file))
else:
    print("No MIDI files found in the folder.")

Files found in: /content/drive/MyDrive/Midis

Extracted MIDI files:
ff7themep.mid
Rachel_Piano_tempofix.mid
lurk_in_dark.mid
bcm.mid
FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Still_Alive-1.mid
FFIII_Edgar_And_Sabin_Piano.mid
sobf.mid
Fiend_Battle_(Piano).mid
ff6shap.mid
FF8_Shuffle_or_boogie_pc.mid
FFIXQuMarshP.mid
roseofmay-piano.mid
ultimafro.mid
Suteki_Da_Ne_(Piano_Version).mid
0fithos.mid
FF4.mid
Kingdom_Hearts_Traverse_Town.mid
figaro.mid
balamb.mid
OTD5YA.mid
dontbeafraid.mid
ff4-fight1.mid
ViviinAlexandria.mid
BlueStone_LastDungeon.mid
sandy.mid
DOS.mid
waltz_de_choco.mid
Ff7-Cinco.mid
relmstheme-piano.mid
thenightmarebegins.mid
Final_Fantasy_Matouyas_Cave_Piano.mid
goldsaucer.mid
Final_Fantasy_7_-_Judgement_Day_Piano.mid
tpirtsd-piano.mid
Oppressed.mid
ultros.mid
ff11_awakening_piano.mid
ff8-lfp.mid
ff7-mainmidi.mid
sera_.mid
mining.mid
Cids.mid
VincentPiano.mid
Finalfantasy6fanfarecomplete.mid
FF3_Battle_(Piano).mid
8.mid
AT.mid
Life_Stream.mid
fortresscondor.mid
ge

In [ ]:
def get_notes(midi_folder_path='/content/drive/MyDrive/Midis'):
    notes = []

    if not os.path.exists(midi_folder_path):
        print(f"Folder not found: {midi_folder_path}")
        return notes

    midi_files = glob.glob(os.path.join(midi_folder_path, '*.mid'))
    if not midi_files:
        print("No MIDI files found in the specified folder.")
        return notes

    print(f"Found {len(midi_files)} MIDI files.")

    for file in midi_files:
        print(f"Parsing {file}")
        try:
            midi = converter.parse(file)
            notes_to_parse = (
                instrument.partitionByInstrument(midi).parts[0].recurse()
                if instrument.partitionByInstrument(midi)
                else midi.flat.notes
            )

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Error parsing {file}: {e}")

    save_path = '/content/drive/MyDrive/data'
    os.makedirs(save_path, exist_ok=True)
    with open(os.path.join(save_path, 'notes'), 'wb') as filepath:
        pickle.dump(notes, filepath)

    print("Notes saved successfully.")
    return notes

In [ ]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(notes))
    note_to_int = {note: num for num, note in enumerate(pitchnames)}
    network_input = [ [note_to_int[note] for note in notes[i:i + sequence_length]] for i in range(len(notes) - sequence_length)]
    network_output = [note_to_int[notes[i + sequence_length]] for i in range(len(notes) - sequence_length)]
    network_input = np.reshape(network_input, (len(network_input), sequence_length, 1)) / float(n_vocab)
    network_output = to_categorical(network_output)
    return network_input, network_output


## Model Development & Training

> Defining and Training the 3-Layered LSTM

In [ ]:
def create_network(network_input, n_vocab):
    model = Sequential([
        LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), recurrent_dropout=0.3, return_sequences=True),
        LSTM(512, return_sequences=True, recurrent_dropout=0.3),
        LSTM(512),
        BatchNorm(),
        Dropout(0.3),
        Dense(256, activation='relu'),
        BatchNorm(),
        Dropout(0.3),
        Dense(n_vocab, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model


In [ ]:
with open('/content/drive/MyDrive/data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)

network_input, _ = prepare_sequences(notes, n_vocab)

model = create_network(network_input, n_vocab)

model.summary()

In [ ]:
def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

In [ ]:
def train(model, network_input, network_output):
    filepath = "/content/drive/MyDrive/weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
    os.makedirs('/content/drive/MyDrive/weights', exist_ok=True)
    checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')


    history = model.fit(
        network_input,
        network_output,
        epochs=120,
        batch_size=128,
        callbacks=[checkpoint]
    )


    final_model_path = "/content/drive/MyDrive/weights/final_model.keras"
    model.save(final_model_path)
    print(f"Model saved to {final_model_path}")

    return history

In [ ]:
train_network()

In [ ]:
from tensorflow.keras.models import load_model


notes = get_notes()
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)


model_path = "/content/drive/MyDrive/weights/final_model.keras"
model = load_model(model_path)

filepath = "/content/drive/MyDrive/weights/weights-improvement-{epoch:02d}-{loss:.4f}-extended.keras"


os.makedirs('/content/drive/MyDrive/weights', exist_ok=True)


checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')

history = model.fit(
    network_input,
    network_output,
    epochs=30,
    batch_size=128,
    callbacks=[checkpoint]
)


final_model_path = "/content/drive/MyDrive/weights/final_model_extended.keras"
model.save(final_model_path)
print(f"Model saved to {final_model_path}")

## Music Generation

> Predicting & Generating Notes, Creating MIDI

In [18]:
def generate_notes(model, network_input, pitchnames, n_vocab, temperature=2, num_notes=200):
    start = np.random.randint(0, len(network_input) - 1)
    int_to_note = {num: note for num, note in enumerate(pitchnames)}
    pattern = network_input[start].tolist()
    prediction_output = []

    pattern = np.array(pattern)


    for i in tqdm(range(num_notes), desc="Notes Generated", ncols=100):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        predictions = model.predict(prediction_input, verbose=0)[0]


        predictions = np.asarray(predictions).astype('float64')
        predictions = np.log(predictions + 1e-7) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))

        index = np.random.choice(range(n_vocab), p=predictions)
        prediction_output.append(int_to_note[index])


        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output


In [12]:
def convert_to_pitch_name(note_number, octave=4):
    note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return f"{note_names[note_number % 12]}{octave}"

In [13]:
def create_midi(prediction_output, output_file='/content/drive/MyDrive/outputf.mid'):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        try:
            if '.' in pattern:

                notes = [note.Note(convert_to_pitch_name(int(n)), storedInstrument=instrument.Piano()) for n in pattern.split('.')]
                if len(notes) > 3:
                    notes = notes[:3]
                output_notes.append(chord.Chord(notes, offset=offset))
            else:
                output_notes.append(note.Note(convert_to_pitch_name(int(pattern)), offset=offset, storedInstrument=instrument.Piano()))

            offset += 0.5
        except (ValueError, TypeError) as e:
            print(f"Error while processing pattern {pattern}: {e}")
            continue

    stream.Stream(output_notes).write('midi', fp=output_file)


In [14]:
def generate_music(temperature, num_notes):
    try:

        with open('/content/drive/MyDrive/data/notes', 'rb') as filepath:
            notes = pickle.load(filepath)

        pitchnames = sorted(set(notes))
        n_vocab = len(pitchnames)


        network_input, _ = prepare_sequences(notes, n_vocab)


        model = load_model('/content/drive/MyDrive/weights/final_model_extended.keras')


        prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, temperature, num_notes)


        output_file = '/content/drive/MyDrive/outputf.mid'
        create_midi(prediction_output, output_file)

        return output_file
    except Exception as e:
        print(f"Error during generation: {e}")
        return f"An error occurred: {str(e)}"

## Interactive UI

> Number of Notes, Temperature Control


In [ ]:
pip install gradio

In [17]:
import gradio as gr

def create_gradio_ui():

    num_notes_slider = gr.Slider(minimum=200, maximum=500, value=200, label="Number of Notes", step=10)
    temperature_slider = gr.Slider(minimum=0.1, maximum=2, value=1, label="Temperature", step=0.1)


    with gr.Blocks() as demo:

        with gr.Row():
            gr.Markdown("# RagaXonic: AI Music Generator")
            gr.Image("/content/logo-Photoroom.png", elem_id="logo", width=200, height=200, interactive=False, show_label=False)

        gr.Markdown("#### A custom 3-layered LSTM which fuses Hindustani and Western music, generating polyphonic MIDI files. Developed by Manas K.")
        gr.Markdown("_May take up to ~150s for generation._")

        with gr.Tab("Generate Music"):
            gr.Markdown("### Generate Music")
            num_notes_slider.render()
            temperature_slider.render()
            generate_button = gr.Button("Generate Music")
            output_midi = gr.File(label="Download MIDI", interactive=False)

            def update_status(temperature, num_notes):

                output_file = generate_music(temperature, num_notes)
                return output_file


            generate_button.click(update_status, inputs=[temperature_slider, num_notes_slider], outputs=[output_midi])

        with gr.Tab("Model Summary"):
            gr.Markdown("### Model Summary")
            gr.Image('/content/model_summary.png', show_label=False)

    return demo

demo = create_gradio_ui()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3432d9349b4d834e98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
